## Excel sheet structure 

In [2]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# First, let's look at the structure of the stations file
stations_file = r"D:\RICAAR\Pr.New.Stations.Selection\OBSERVATIONS\Selected.Pr.Stations.locations.xlsx"
stations_df = pd.read_excel(stations_file)

# Print column names to see what we're working with
print("Columns in stations file:")
print(stations_df.columns.tolist())
print("\nFirst few rows of the data:")
print(stations_df.head())

Columns in stations file:
['STATION_ID', 'STATION_NAME', 'PALESTINE_NORTH', 'PALESTINE_EAST', 'Longitude', 'Latitude', 'ELEVATION', 'TYPE', 'BASIN', 'PERIOD', 'NOTE']

First few rows of the data:
  STATION_ID                     STATION_NAME  PALESTINE_NORTH  \
0     H 0001          H4 EVAP.ST (METEO DEPT)          1216000   
1     H 0003                  KH.UM RUJEIM TO          1228800   
2     H 0002                      TARABEEL TO          1237899   
3     F 0007                      AL-WISAD TO          1144800   
4     AD0032  BAQURA MET.STATION (METEO DEPT)          1224300   

   PALESTINE_EAST  Longitude   Latitude  ELEVATION    TYPE         BASIN  \
0          450500  38.195443  32.502641      755.0   DAILY        HAMMAD   
1          486800  38.585841  32.608084        NaN  ANNAUL        HAMMAD   
2          508491  38.820090  32.683531        NaN  ANNAUL        HAMMAD   
3          435000  38.010807  31.865164      700.0   DAILY        HAMMAD   
4          206300  35.59698

In [3]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Define paths
base_path = r"D:\RICAAR\Pr.New.Stations.Selection\validation_results3"
stations_file = r"D:\RICAAR\Pr.New.Stations.Selection\OBSERVATIONS\Selected.Pr.Stations.locations.xlsx"
output_dir = r"D:\RICAAR\Pr.New.Stations.Selection\validation.according.to.basin.2"

# Create output directory if it doesn't exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

# List of models
models = [
    'CMCC-CM2-SR5',
    'CNRM-ESM2-1',
    'EC-Earth3-Veg',
    'IPSL-CM6A-LR',
    'NorESM2-MM',
    'MPI-ESM1-2-LR'
]

# Read stations and their basin information
stations_df = pd.read_excel(stations_file)

def process_model_data(model_name):
    """Process validation metrics for a single model"""
    model_path = os.path.join(base_path, model_name, "validation_heatmaps")
    
    # Read all metrics
    metrics = {}
    for metric in ['r', 'NSE', 'PBIAS', 'RMSE', 'MAE']:
        file_path = os.path.join(model_path, f'{metric}_values.xlsx')
        if os.path.exists(file_path):
            metrics[metric] = pd.read_excel(file_path)
    
    return metrics

def calculate_basin_metrics(metrics, basin_stations):
    """Calculate average metrics for a basin"""
    basin_metrics = {}
    
    for metric_name, metric_data in metrics.items():
        # Convert Station_ID to match format in stations_df if needed
        basin_values = metric_data[metric_data['Station_ID'].isin(basin_stations)]['Value']
        
        basin_metrics[f'{metric_name}_mean'] = basin_values.mean()
        basin_metrics[f'{metric_name}_std'] = basin_values.std()
        basin_metrics[f'{metric_name}_min'] = basin_values.min()
        basin_metrics[f'{metric_name}_max'] = basin_values.max()
        basin_metrics[f'{metric_name}_count'] = len(basin_values)
    
    return basin_metrics

# Process all models
results = {}
for model in models:
    print(f"Processing model: {model}")
    
    # Get model metrics
    model_metrics = process_model_data(model)
    
    # Create detailed results for each basin
    basin_results = {}
    basin_summaries = []
    
    for basin in stations_df['BASIN'].unique():
        # Get stations for this basin
        basin_stations = stations_df[stations_df['BASIN'] == basin]['STATION_ID'].tolist()
        
        # Get detailed station data for this basin
        basin_detail = pd.DataFrame()
        for metric, data in model_metrics.items():
            basin_data = data[data['Station_ID'].isin(basin_stations)]
            if len(basin_detail) == 0:
                basin_detail['Station_ID'] = basin_data['Station_ID']
                basin_detail['Station_Name'] = basin_data['Station_Name']
            basin_detail[f'{metric}'] = basin_data['Value']
        
        if not basin_detail.empty:  # Only add if we have data for this basin
            basin_results[basin] = basin_detail
            
            # Calculate basin summary metrics
            summary = calculate_basin_metrics(model_metrics, basin_stations)
            summary['Basin'] = basin
            basin_summaries.append(summary)
    
    results[model] = {
        'basin_details': basin_results,
        'basin_summary': pd.DataFrame(basin_summaries)
    }

# Save results for each model separately
for model in results:
    model_output_file = os.path.join(output_dir, f'{model}_Basin_Validation.xlsx')
    with pd.ExcelWriter(model_output_file) as writer:
        # Save basin details
        for basin, data in results[model]['basin_details'].items():
            sheet_name = f"{basin[:31]}"  # Excel sheet names limited to 31 chars
            data.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Save basin summary
        results[model]['basin_summary'].to_excel(writer, sheet_name='Basin_Summary', index=False)

# Create and save overall comparison
overall_comparison = []
for model in results:
    model_summary = results[model]['basin_summary'].copy()
    model_summary['Model'] = model
    overall_comparison.append(model_summary)

overall_df = pd.concat(overall_comparison, ignore_index=True)
overall_df.to_excel(os.path.join(output_dir, 'Overall_Basin_Comparison.xlsx'), index=False)

print("\nResults saved to:", output_dir)
print("\nFiles created:")
for model in models:
    print(f"- {model}_Basin_Validation.xlsx")
print("- Overall_Basin_Comparison.xlsx")

Processing model: CMCC-CM2-SR5
Processing model: CNRM-ESM2-1
Processing model: EC-Earth3-Veg
Processing model: IPSL-CM6A-LR
Processing model: NorESM2-MM
Processing model: MPI-ESM1-2-LR

Results saved to: D:\RICAAR\Pr.New.Stations.Selection\validation.according.to.basin.2

Files created:
- CMCC-CM2-SR5_Basin_Validation.xlsx
- CNRM-ESM2-1_Basin_Validation.xlsx
- EC-Earth3-Veg_Basin_Validation.xlsx
- IPSL-CM6A-LR_Basin_Validation.xlsx
- NorESM2-MM_Basin_Validation.xlsx
- MPI-ESM1-2-LR_Basin_Validation.xlsx
- Overall_Basin_Comparison.xlsx


## the above code worked but the below is a modification to eleminate some stations 

In [4]:
import pandas as pd
import numpy as np
import os
from pathlib import Path

# Define paths
base_path = r"D:\RICAAR\Pr.New.Stations.Selection\validation_results3"
stations_file = r"D:\RICAAR\Pr.New.Stations.Selection\OBSERVATIONS\Selected.Pr.Stations.locations.xlsx"
output_dir = r"D:\RICAAR\Pr.New.Stations.Selection\validation.according.to.basin.2"

# Create output directory if it doesn't exist
Path(output_dir).mkdir(parents=True, exist_ok=True)

# List of models
models = [
    'CMCC-CM2-SR5',
    'CNRM-ESM2-1',
    'EC-Earth3-Veg',
    'IPSL-CM6A-LR',
    'NorESM2-MM',
    'MPI-ESM1-2-LR'
]

# Stations to exclude
excluded_stations = ['F 0007', 'DA0007', 'CF0008', 'ED0004', 'ED0012']

# Read stations and their basin information
stations_df = pd.read_excel(stations_file)
# Remove excluded stations
stations_df = stations_df[~stations_df['STATION_ID'].isin(excluded_stations)]

def process_model_data(model_name):
    """Process validation metrics for a single model"""
    model_path = os.path.join(base_path, model_name, "validation_heatmaps")
    
    # Read all metrics
    metrics = {}
    for metric in ['r', 'NSE', 'PBIAS', 'RMSE', 'MAE']:
        file_path = os.path.join(model_path, f'{metric}_values.xlsx')
        if os.path.exists(file_path):
            metrics[metric] = pd.read_excel(file_path)
            # Remove excluded stations from metrics data
            metrics[metric] = metrics[metric][~metrics[metric]['Station_ID'].isin(excluded_stations)]
    
    return metrics

def calculate_basin_metrics(metrics, basin_stations):
    """Calculate average metrics for a basin"""
    basin_metrics = {}
    
    for metric_name, metric_data in metrics.items():
        basin_values = metric_data[metric_data['Station_ID'].isin(basin_stations)]['Value']
        
        basin_metrics[f'{metric_name}_mean'] = basin_values.mean()
        basin_metrics[f'{metric_name}_std'] = basin_values.std()
        basin_metrics[f'{metric_name}_min'] = basin_values.min()
        basin_metrics[f'{metric_name}_max'] = basin_values.max()
        basin_metrics[f'{metric_name}_count'] = len(basin_values)
    
    return basin_metrics

# Process all models
results = {}
for model in models:
    print(f"Processing model: {model}")
    
    # Get model metrics
    model_metrics = process_model_data(model)
    
    # Create detailed results for each basin
    basin_results = {}
    basin_summaries = []
    
    for basin in stations_df['BASIN'].unique():
        # Get stations for this basin
        basin_stations = stations_df[stations_df['BASIN'] == basin]['STATION_ID'].tolist()
        
        # Get detailed station data for this basin
        basin_detail = pd.DataFrame()
        for metric, data in model_metrics.items():
            basin_data = data[data['Station_ID'].isin(basin_stations)]
            if len(basin_detail) == 0:
                basin_detail['Station_ID'] = basin_data['Station_ID']
                basin_detail['Station_Name'] = basin_data['Station_Name']
            basin_detail[f'{metric}'] = basin_data['Value']
        
        if not basin_detail.empty:  # Only add if we have data for this basin
            basin_results[basin] = basin_detail
            
            # Calculate basin summary metrics
            summary = calculate_basin_metrics(model_metrics, basin_stations)
            summary['Basin'] = basin
            basin_summaries.append(summary)
    
    results[model] = {
        'basin_details': basin_results,
        'basin_summary': pd.DataFrame(basin_summaries)
    }

# Save results for each model separately
for model in results:
    model_output_file = os.path.join(output_dir, f'{model}_Basin_Validation.xlsx')
    with pd.ExcelWriter(model_output_file) as writer:
        # Save basin details
        for basin, data in results[model]['basin_details'].items():
            sheet_name = f"{basin[:31]}"  # Excel sheet names limited to 31 chars
            data.to_excel(writer, sheet_name=sheet_name, index=False)
        
        # Save basin summary
        results[model]['basin_summary'].to_excel(writer, sheet_name='Basin_Summary', index=False)

# Create and save overall comparison
overall_comparison = []
for model in results:
    model_summary = results[model]['basin_summary'].copy()
    model_summary['Model'] = model
    overall_comparison.append(model_summary)

overall_df = pd.concat(overall_comparison, ignore_index=True)
overall_df.to_excel(os.path.join(output_dir, 'Overall_Basin_Comparison.xlsx'), index=False)

# Print summary of excluded stations
print("\nExcluded stations:")
for station in excluded_stations:
    station_info = stations_df[stations_df['STATION_ID'] == station]
    if not station_info.empty:
        print(f"- {station}: {station_info['BASIN'].iloc[0]} basin")

print("\nResults saved to:", output_dir)
print("\nFiles created:")
for model in models:
    print(f"- {model}_Basin_Validation.xlsx")
print("- Overall_Basin_Comparison.xlsx")

Processing model: CMCC-CM2-SR5
Processing model: CNRM-ESM2-1
Processing model: EC-Earth3-Veg
Processing model: IPSL-CM6A-LR
Processing model: NorESM2-MM
Processing model: MPI-ESM1-2-LR

Excluded stations:

Results saved to: D:\RICAAR\Pr.New.Stations.Selection\validation.according.to.basin.2

Files created:
- CMCC-CM2-SR5_Basin_Validation.xlsx
- CNRM-ESM2-1_Basin_Validation.xlsx
- EC-Earth3-Veg_Basin_Validation.xlsx
- IPSL-CM6A-LR_Basin_Validation.xlsx
- NorESM2-MM_Basin_Validation.xlsx
- MPI-ESM1-2-LR_Basin_Validation.xlsx
- Overall_Basin_Comparison.xlsx


## best model for basins 

In [5]:
import pandas as pd
import numpy as np
import os

# Read the overall comparison file
comparison_file = r"D:\RICAAR\Pr.New.Stations.Selection\validation.according.to.basin.2\Overall_Basin_Comparison.xlsx"
df = pd.read_excel(comparison_file)

def rank_models_by_basin():
    """Rank models for each basin based on multiple criteria"""
    
    # Define which metrics are better when higher or lower
    higher_better = ['r_mean', 'NSE_mean']
    lower_better = ['RMSE_mean', 'MAE_mean', 'PBIAS_mean']  # Using absolute value for PBIAS
    
    # Add absolute PBIAS column
    df['abs_PBIAS_mean'] = df['PBIAS_mean'].abs()
    
    # Create ranking dataframe
    rankings = pd.DataFrame()
    
    for basin in df['Basin'].unique():
        basin_data = df[df['Basin'] == basin].copy()
        
        # Rank for each metric (1 is best)
        for metric in higher_better:
            basin_data[f'{metric}_rank'] = basin_data[metric].rank(ascending=False)
            
        for metric in lower_better:
            if metric == 'PBIAS_mean':
                basin_data[f'{metric}_rank'] = basin_data['abs_PBIAS_mean'].rank()
            else:
                basin_data[f'{metric}_rank'] = basin_data[metric].rank()
        
        # Calculate average rank
        rank_columns = [col for col in basin_data.columns if col.endswith('_rank')]
        basin_data['avg_rank'] = basin_data[rank_columns].mean(axis=1)
        
        # Sort by average rank
        basin_data = basin_data.sort_values('avg_rank')
        
        # Add to rankings
        rankings = pd.concat([rankings, basin_data])
    
    return rankings

def create_performance_summary(rankings):
    """Create a detailed performance summary"""
    
    summary = []
    
    for basin in rankings['Basin'].unique():
        basin_data = rankings[rankings['Basin'] == basin].copy()
        
        # Get best model
        best_model = basin_data.iloc[0]
        
        summary.append({
            'Basin': basin,
            'Best_Model': best_model['Model'],
            'r_value': best_model['r_mean'],
            'NSE_value': best_model['NSE_mean'],
            'PBIAS_value': best_model['PBIAS_mean'],
            'RMSE_value': best_model['RMSE_mean'],
            'MAE_value': best_model['MAE_mean'],
            'Average_Rank': best_model['avg_rank']
        })
    
    return pd.DataFrame(summary)

# Generate rankings and summary
rankings = rank_models_by_basin()
summary = create_performance_summary(rankings)

# Save results
output_dir = r"D:\RICAAR\Pr.New.Stations.Selection\validation.according.to.basin.2"
rankings.to_excel(os.path.join(output_dir, 'Model_Rankings_by_Basin.xlsx'), index=False)
summary.to_excel(os.path.join(output_dir, 'Best_Models_Summary.xlsx'), index=False)

# Print summary
print("\nBest Model for Each Basin:")
print("=" * 50)
for _, row in summary.iterrows():
    print(f"\nBasin: {row['Basin']}")
    print(f"Best Model: {row['Best_Model']}")
    print(f"Performance Metrics:")
    print(f"  R-value: {row['r_value']:.3f}")
    print(f"  NSE: {row['NSE_value']:.3f}")
    print(f"  PBIAS: {row['PBIAS_value']:.3f}%")
    print(f"  RMSE: {row['RMSE_value']:.3f}")
    print(f"  MAE: {row['MAE_value']:.3f}")
    print(f"  Average Rank: {row['Average_Rank']:.2f}")


Best Model for Each Basin:

Basin: HAMMAD
Best Model: MPI-ESM1-2-LR
Performance Metrics:
  R-value: 0.926
  NSE: -0.657
  PBIAS: 9.603%
  RMSE: 3.294
  MAE: 2.732
  Average Rank: 1.20

Basin: JORDAN VALLY
Best Model: CNRM-ESM2-1
Performance Metrics:
  R-value: 0.976
  NSE: 0.724
  PBIAS: 16.097%
  RMSE: 12.159
  MAE: 10.505
  Average Rank: 1.60

Basin: N.R.S.W
Best Model: NorESM2-MM
Performance Metrics:
  R-value: 0.991
  NSE: 0.835
  PBIAS: -14.528%
  RMSE: 14.759
  MAE: 12.635
  Average Rank: 1.00

Basin: YARMOUK
Best Model: CNRM-ESM2-1
Performance Metrics:
  R-value: 0.951
  NSE: 0.646
  PBIAS: 7.589%
  RMSE: 10.105
  MAE: 7.737
  Average Rank: 1.80

Basin: AMMAN ZARQA
Best Model: CMCC-CM2-SR5
Performance Metrics:
  R-value: 0.973
  NSE: 0.454
  PBIAS: 1.922%
  RMSE: 14.644
  MAE: 11.844
  Average Rank: 2.00

Basin: MUJIB
Best Model: CMCC-CM2-SR5
Performance Metrics:
  R-value: 0.953
  NSE: 0.159
  PBIAS: 0.184%
  RMSE: 10.698
  MAE: 9.047
  Average Rank: 1.60

Basin: AZRAQ
Best Mo